In [ ]:
from crewai import Agent, LLM
from dotenv import load_dotenv
import os
load_dotenv()  # Load environment variables from .env file
#os.environ["OPENAI_API_KEY"] = os.getenv("GEMINI_API_KEY")


llm = LLM(model="gemini/gemini-2.0-flash", verbose=True, temperature=0.5,
          api_key=os.getenv("GEMINI_API_KEY"))

## Tasks Guardrails

In [ ]:
def validate_summary_length(task_output):
    try:
        print("Validating summary length")
        task_str_output = str(task_output)
        total_words = len(task_str_output.split())

        print(f"Word count: {total_words}")

        if total_words > 150:
            print("Summary exceeds 150 words")
            return (False, f"Summary exceeds 150 words. Word count: {total_words}")

        if total_words == 0:
            print("Summary is empty")
            return (False, "Generated summary is empty.")

        print("Summary is valid")
        return (True, task_output)

    except Exception as e:
        print("Validation system error")
        return (False, f"Validation system error: {str(e)}")


In [ ]:
from crewai import Task, Agent

summary_agent = Agent(
    role="Summary Agent",
    goal="Summarize the research paper 'Convolutional Neural Networks' in 150 words.",
    backstory="You are a specialized agent that summarizes research papers.",
    verbose=True,
    llm=llm
)

summary_task = Task(
    description="Summarize a research paper in 150 words.",
    expected_output="A concise research summary 150 words.",
    agent=summary_agent,
    guardrail=validate_summary_length,

    max_retries=3
)


In [ ]:
from crewai import Crew

summary_crew = Crew(
    agents=[summary_agent],
    tasks=[summary_task],
    verbose=True
)

result = summary_crew.kickoff()

### Ask the agent to write a 200 word summary instead and notice the guardrail output:

In [ ]:
from crewai import Task, Agent

summary_agent = Agent(
    role="Summary Agent",
    goal="Summarize the research paper 'Convolutional Neural Networks' in 200 words.",
    backstory="You are a specialized agent that summarizes research papers.",
    verbose=True,
    llm=llm
)

summary_task = Task(
    description="Summarize a research paper in 200 words.",
    expected_output="A concise research summary 200 words.",
    agent=summary_agent,
    guardrail=validate_summary_length,
    max_retries=3
)


In [ ]:
from crewai import Crew

summary_crew = Crew(
    agents=[summary_agent],
    tasks=[summary_task],
    verbose=True
)

result = summary_crew.kickoff()

## Tasks Guardrails with Pydantic

In [ ]:
from pydantic import BaseModel

class ResearchReport(BaseModel):
    """Represents a structured research report"""
    title: str
    summary: str
    key_findings: list[str]

In [ ]:
import json
from typing import Tuple, Any

def validate_json_report(result: str) -> Tuple[bool, Any]:
    """Ensures AI-generated output is valid JSON with required fields."""
    try:
        # Parse JSON output
        data = json.loads(result.pydantic.model_dump_json())

        # Check required fields
        if "title" not in data or "summary" not in data or "key_findings" not in data:
            return (False, "Missing required fields: title, summary, or key_findings.")

        return (True, result)  # JSON is valid
    except json.JSONDecodeError:
        return (False, "Invalid JSON format. Please ensure correct syntax.")


In [ ]:
from crewai import Agent

# Create the AI Agent
research_report_agent = Agent(
    role="Research Analyst",
    goal="Generate structured JSON reports for research papers",
    backstory="You are an expert in technical writing and structured reporting.",
    verbose=False,
    llm=llm
)


In [ ]:
from crewai import Task

research_report_task = Task(
    description="Generate a structured research report in valid JSON format.",
    expected_output="A valid JSON object containing 'title', 'summary', and 'key_findings'.",
    agent=research_report_agent,
    output_pydantic=ResearchReport,  # Ensures structured output
    guardrail=validate_json_report,  # Validate output before passing to next step
    max_retries=3  # Allow up to 3 retries if validation fails
)

In [ ]:
from crewai import Crew

research_crew = Crew(
    agents=[research_report_agent],
    tasks=[research_report_task],
    verbose=True  # Display execution details
)


In [ ]:
result = research_crew.kickoff()

# Display the validated JSON output
print("Final Research Report:", result.pydantic)

## Getting Structured Consistent Outputs from Tasks

In [ ]:
from pydantic import BaseModel

class ResearchFindings(BaseModel):
    """Structured research report output"""
    title: str
    key_findings: list[str]

class AnalysisSummary(BaseModel):
    """Structured summary of research findings"""
    insights: list[str]
    key_takeaways: str


In [ ]:
from crewai import Agent, LLM

# AI Agents
research_agent = Agent(
    role="AI Researcher",
    goal="Find and summarize the latest AI advancements",
    backstory="You are an expert AI researcher who stays up to date with the latest innovations.",
    verbose=True,
    llm=llm
)

analysis_agent = Agent(
    role="AI Analyst",
    goal="Analyze AI research findings and extract key insights",
    backstory="You are a data analyst who extracts valuable insights from research data.",
    verbose=True,
    llm=llm
)

writer_agent = Agent(
    role="Tech Writer",
    goal="Write a well-structured blog post on AI trends",
    backstory="You are a technology writer skilled at transforming complex AI research into readable content.",
    verbose=True,
    llm=llm
)


In [ ]:
from crewai import Task

# Step 1: Research Task
research_task = Task(
    description="Find and summarize the latest AI advancements",
    expected_output="A structured list of recent AI breakthroughs",
    agent=research_agent,
    output_pydantic=ResearchFindings  # Structured output
)

# Step 2: Analysis Task (References Research Task Output)
analysis_task = Task(
    description="Analyze AI research findings and extract key insights",
    expected_output="A structured summary with key takeaways",
    agent=analysis_agent,
    output_pydantic=AnalysisSummary,
    context=[research_task]  # Receives output from research_task
)

# Step 3: Blog Writing Task (References Both Research and Analysis)
blog_writing_task = Task(
    description="Write a detailed blog post about AI trends",
    expected_output="A well-structured blog post",
    agent=writer_agent,
    context=[research_task, analysis_task]  # Uses both research and analysis outputs
)


In [ ]:
from crewai import Crew

ai_research_crew = Crew(
    agents=[research_agent, analysis_agent, writer_agent],
    tasks=[research_task, analysis_task, blog_writing_task],
    verbose=True
)

# Execute the workflow
result = ai_research_crew.kickoff()

# Print the final blog post
print("\n=== Generated Blog Post ===")
print(result.raw)


In [ ]:
analysis_task.output

In [ ]:
analysis_task.output.pydantic

## Async Execution

In [ ]:
from pydantic import BaseModel

class AIResearchFindings(BaseModel):
    """Represents structured research on AI breakthroughs."""
    title: str
    key_findings: list[str]

class AIRegulationFindings(BaseModel):
    """Represents structured research on AI regulations."""
    region: str
    key_policies: list[str]

class FinalAIReport(BaseModel):
    """Combines AI research & regulation analysis into a report."""
    executive_summary: str
    key_trends: list[str]


In [ ]:
from crewai import Agent

# Researcher for AI breakthroughs
research_agent = Agent(
    role="AI Researcher",
    goal="Find and summarize the latest AI breakthroughs",
    backstory="An expert AI researcher who tracks technological advancements.",
    verbose=True,
    llm=llm
)

# Analyst for AI regulations
regulation_agent = Agent(
    role="AI Policy Analyst",
    goal="Analyze global AI regulations and summarize policies",
    backstory="A government policy expert specializing in AI ethics and laws.",
    verbose=True,
    llm=llm
)

# Writer for the final AI report
writer_agent = Agent(
    role="AI Report Writer",
    goal="Write a structured report combining AI breakthroughs and regulations",
    backstory="A professional technical writer who crafts AI research reports.",
    verbose=True,
    llm=llm
)


In [ ]:
from crewai import Task

# Task 1: AI Breakthroughs Research (Asynchronous)
research_ai_task = Task(
    description="Research the latest AI advancements and summarize key breakthroughs.",
    expected_output="A structured list of AI breakthroughs.",
    agent=research_agent,
    output_pydantic=AIResearchFindings,
    async_execution=True  # Runs asynchronously
)

# Task 2: AI Regulation Analysis (Asynchronous)
research_regulation_task = Task(
    description="Analyze the latest AI regulations worldwide and summarize key policies.",
    expected_output="A structured summary of AI regulations by region.",
    agent=regulation_agent,
    output_pydantic=AIRegulationFindings,
    async_execution=True  # Runs asynchronously
)

# Task 3: Generate AI Research Report (Depends on the first two tasks)
generate_report_task = Task(
    description="Write a structured report summarizing AI breakthroughs and regulations.",
    expected_output="A final AI report summarizing both aspects.",
    agent=writer_agent,
    output_pydantic=FinalAIReport,
    context=[research_ai_task, research_regulation_task]  # Waits for these tasks to complete
)


In [ ]:
from crewai import Crew

ai_research_crew = Crew(
    agents=[research_agent, regulation_agent, writer_agent],
    tasks=[research_ai_task, research_regulation_task, generate_report_task],
    verbose=True
)

# Execute the workflow
result = ai_research_crew.kickoff()

# Print the final AI report
print("\n=== Generated AI Report ===")
print(result.raw)

## Callbacks

In [ ]:
from crewai import Agent

# AI Researcher Agent
research_agent = Agent(
    role="AI News Researcher",
    goal="Find and summarize the latest AI news from trusted sources",
    backstory="You are a dedicated AI journalist who follows the latest advancements in artificial intelligence.",
    verbose=False,
    llm=llm
)


In [ ]:
def notify_team(output):

    print(f"""Task Completed!
              Task: {output.description}
              Output Summary: {output.summary}""")

    with open("latest_ai_news.txt", "w") as f:
        f.write(f"Task: {output.description}\n")
        f.write(f"Output Summary: {output.summary}\n")
        f.write(f"Full Output: {output.raw}\n")
    
    print("News summary saved to latest_ai_news.txt")

In [ ]:
from crewai import Task

research_news_task = Task(
    description="Find and summarize the latest AI breakthroughs from the last week.",
    expected_output="A structured summary of AI news headlines.",
    agent=research_agent,
    callback=notify_team  # Calls the function after task completion
)


In [ ]:
from crewai import Crew

ai_news_crew = Crew(
    agents=[research_agent],
    tasks=[research_news_task],
    verbose=False
)

# Execute the workflow
result = ai_news_crew.kickoff()


## Hierarchical Process

In [ ]:
from crewai import Agent, LLM

llm = LLM(model="gpt-4o", api_key=os.environ["OPENAI_API_KEY"])

# research a new project idea, do the research on market demand, risk, and potential return on investment.

# Define the Manager AI
manager_agent = Agent(
    role="Project Research Manager",
    goal="Oversee the project research and ensure timely, high-quality responses.",
    backstory="""An experienced project manager responsible
                 for ensuring project research.""",
    allow_delegation=True,
    verbose=True,
    llm=llm
)

# Define the Technical Support AI
market_demand_agent = Agent(
    role="Market Demand Analyst",
    goal="Write market demand content.",
    backstory="""A skilled market demand analyst who
                 writes market demand content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)

# Define the Fiction Writer AI
risk_analysis_agent = Agent(
    role="Risk Analysis Analyst",
    goal="Write risk analysis content.",
    backstory="""A risk analysis analyst who
                 writes fiction content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)

# Define the Fiction Writer AI
return_on_investment_agent = Agent(
    role="Return on Investment Analyst",
    goal="Write return on investment content.",
    backstory="""A return on investment analyst who
                writes return on investment content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)


In [ ]:
from crewai import Task

manager_task = Task(
    description="""Oversee the project research on {project_title} and ensure timely, high-quality responses.""",
    expected_output="A manager-approved response ready to be sent as an article on {project_title}.",
    agent=manager_agent, 
)

market_demand_task = Task(
    description="""Analyze the market demand for the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=market_demand_agent, 
)

risk_analysis_task = Task(
    description="""Analyze the risk of the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=risk_analysis_agent, 
)

return_on_investment_task = Task(
    description="""Analyze the return on investment of the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=return_on_investment_agent, 
)

final_report_task = Task(
    description="""Review the final responses from the 
                   market demand, risk analysis, and return on investment agents
                   and create a final report.""",
    expected_output="""A comprehensive report on the project title '{project_title}'
    containing the market demand, risk analysis, and return on investment.""",
    agent=manager_agent,
)

In [ ]:
from crewai import Crew, Process

project_research_crew = Crew(
    agents=[market_demand_agent, risk_analysis_agent, return_on_investment_agent],
    
    tasks=[market_demand_task, risk_analysis_task, return_on_investment_task, final_report_task],
    
    manager_agent=manager_agent,
    
    process=Process.hierarchical,
    
    verbose=True,
)

In [ ]:
inputs = {"project_title": "Amazon INC AMZN"}

result = project_research_crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)

## Human input

In [ ]:
from crewai import Agent, LLM

llm = LLM(model="gemini/gemini-2.0-flash")

# AI Researcher Agent
researcher_agent = Agent(
    role="Senior AI Researcher",
    goal="Discover and summarize the latest trends in AI and technology.",
    backstory="An expert in AI research who tracks emerging trends and their real-world applications.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# AI Content Strategist Agent
content_strategist_agent = Agent(
    role="Tech Content Strategist",
    goal="Transform AI research insights into compelling blog content.",
    backstory="An experienced tech writer who makes AI advancements accessible to a broad audience.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [ ]:
from crewai import Task

# Step 1: AI Research with Human Oversight
ai_research_task = Task(
    description=(
        "Conduct a deep analysis of AI trends in 2025. Identify key innovations, breakthroughs, and market shifts. "
        "Before finalizing, ask a human reviewer for feedback to refine the report."
    ),
    expected_output="A structured research summary covering AI advancements in 2025.",
    agent=researcher_agent,
    human_input=True  # Requires human validation before finalizing the answer
)

# Step 2: AI-Generated Blog Post with Human Review
blog_post_task = Task(
    description=(
        "Using insights from the AI Researcher, create an engaging blog post summarizing key AI advancements. "
        "Ensure the post is informative and accessible. Before finalizing, ask a human reviewer for approval."
    ),
    expected_output="A well-structured blog post about AI trends in 2025.",
    agent=content_strategist_agent,
    human_input=True  # Requires human approval before publishing
)

In [ ]:
from crewai import Crew

ai_research_crew = Crew(
    agents=[researcher_agent, content_strategist_agent],  
    tasks=[ai_research_task, blog_post_task],  
    verbose=True,  
)


In [ ]:
# Execute the workflow
result = ai_research_crew.kickoff()

# Display the final validated research output
print("\n=== Final AI Research Report ===")
print(result.raw)


In [ ]:
from IPython.display import Markdown
Markdown(result.raw)

In [ ]:
import os
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
load_dotenv()

llm = LLM(model="gemini/gemini-2.0-flash")

# Loading Tools
search_tool = SerperDevTool()

# Define your agents with roles, goals, tools, and additional attributes
researcher = Agent(
    role='Senior Research Analyst',
    goal='Uncover cutting-edge developments in AI and data science',
    backstory=(
        "You are a Senior Research Analyst at a leading tech think tank. "
        "Your expertise lies in identifying emerging trends and technologies in AI and data science. "
        "You have a knack for dissecting complex data and presenting actionable insights."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=llm
)
writer = Agent(
    role='Tech Content Strategist',
    goal='Craft compelling content on tech advancements',
    backstory=(
        "You are a renowned Tech Content Strategist, known for your insightful and engaging articles on technology and innovation. "
        "With a deep understanding of the tech industry, you transform complex concepts into compelling narratives."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=llm
)


# Create tasks for your agents
task1 = Task(
    description=(
        "Conduct a comprehensive analysis of the latest advancements in AI in 2025. "
        "Identify key trends, breakthrough technologies, and potential industry impacts. "
        "Compile your findings in a detailed report. "
        "Make sure to check with a human if the draft is good before finalizing your answer."
    ),
    expected_output='A comprehensive full report on the latest AI advancements in 2025, leave nothing out',
    agent=researcher,
    human_input=True
)

task2 = Task(
    description=(
        "Using the insights from the researcher\'s report, develop an engaging blog post that highlights the most significant AI advancements. "
        "Your post should be informative yet accessible, catering to a tech-savvy audience. "
        "Aim for a narrative that captures the essence of these breakthroughs and their implications for the future."
    ),
    expected_output='A compelling 3 paragraphs blog post formatted as markdown about the latest AI advancements in 2025',
    agent=writer,
    human_input=True
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True,

)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)